<a href="https://colab.research.google.com/github/kristianJW54/Small_projects/blob/main/basic_login_program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Aim for the program:

1. Run a main() function which is a choose a step log in process
2. Choose log in or create account
3. If log in: then input username which checks against the database, input password which checks against pattern -> returns a user instance
4. If create account: input username and checks if already taken, input password and choose name and surname -> returns user instance


In [ ]:
import re
import os
import csv

class Authentication:
    @classmethod
    def validate_password(cls, password):
        password_length = 5  # Default password length
        pattern = cls.password_pattern(password_length)
        pw = re.fullmatch(pattern, password)
        if pw:
            print("Valid Password")
            return True
        else:
            errors = []
            if len(password) != password_length:
                errors.append(f"Password must be {password_length} characters long")
            if not re.search(r"[A-Z]", password):
                errors.append("Password must contain at least one uppercase letter")
            if not re.search(r"\d", password):
                errors.append("Password must contain at least one digit")
            if not re.search(r"[@$!%*?&]", password):
                errors.append("Password must contain at least one special character")
            print("\n".join(errors))
            return False

    @staticmethod
    def password_pattern(length):
        return rf"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d)(?=.*[@$!%*?&])[A-Za-z\d@$!%*?&]{{{length},}}$"

    @classmethod
    def validate_username(cls, username, db):
        if not username:
            print("Username Not Specified")
            return False

        if isinstance(db, Database):
            db = db.path

        try:
            with open(db, "r") as file:
                reader = csv.DictReader(file)
                for row in reader:
                    if row.get("Username") == username:
                        print("Username already exists.")
                        return False

            print("Valid Username")
            return True
        except Exception as e:
            print(f"Error reading from database: {e}")
            return False

    @classmethod
    def retrieve_username(cls, username, db):
        if not username:
            print("No Username Passed")

        if isinstance(db, Database):
            db = db.path

        try:
            with open(db, "r") as file:
                reader = csv.DictReader(file)
                for row in reader:
                    if row.get("Username") == username:
                        yield row  # Yield the matching row
        except Exception as e:
            print(f"Error retrieving username: {e}")

    @classmethod
    def check_password(cls, password, user_data):
        if not password:
            print("No Password Passed")

        if password == user_data.get("Password"):
            return True
        else:
            print("Incorrect Password")
            return False

    @classmethod
    def login(cls, db):
        if isinstance(db, Database):
            db = db.path

        while True:
            username = input("Enter Username: ")
            try:
                user_data = next(cls.retrieve_username(username, db), None)  # Get the first matching user data
                if user_data:
                    password = input("Enter Password: ").strip()
                    if cls.check_password(password, user_data):
                        print("You're In")
                        # Initialize a user instance with the retrieved user data
                        user_instance = User.initialise_user(user_data)
                        return user_instance
                    else:
                        print("Invalid password. Please try again.")
            except StopIteration:
                print("No user found with that username. Please try again.")



class Database:
    def __init__(self, path):
        self.path = path

    @classmethod
    def connect(cls, path):
        if path and os.path.exists(path):
            print(f"Database {path} ---> connected...")
            return cls(path)
        else:
            print("No valid path specified or database file does not exist.")
            return None

    def write_to_database(self, data):
        try:
            # Load existing data from the file
            existing_data = []
            if os.path.isfile(self.path):
                with open(self.path, "r") as file:
                    reader = csv.DictReader(file)
                    existing_data = list(reader)

            # Append the new data to the existing data
            existing_data.append(data)

            # Write the updated data back to the file
            fieldnames = list(data.keys())
            with open(self.path, "w", newline="") as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(existing_data)

        except Exception as e:
            print(f"Error writing to database: {e}")


class User:
    def __init__(self, user_data):
        self.user_data = user_data

    @classmethod
    def initialise_user(cls, user_data):
        return cls(user_data)

    @classmethod
    def create_admin(cls, db):
        if isinstance(db, Database):
            user_data = {
                "Username": None,
                "Password": None,
                "name": None,
                "surname": None,
                "Level": "Admin"
            }

            username = input("Create Username: ")
            if not Authentication.validate_username(username, db):
                print("Username is not valid or already exists.")
                return None
            user_data["Username"] = username

            password = input("Create Password: ")
            if not Authentication.validate_password(password):
                print("Password is not valid.")
                return None
            user_data["Password"] = password

            name = input("Input Your Name: ")
            if not name:
                print("Name cannot be empty.")
                return None
            user_data["name"] = name

            surname = input("Input Your Surname: ")
            if not surname:
                print("Surname cannot be empty.")
                return None
            user_data["surname"] = surname

            db.write_to_database(user_data)

            return cls(user_data)

    # Can add other user levels to intialise



# Write the test_data to a CSV file
test_data = [
    {"Username": "billy54", "Password": "billyB54@", "name": "Billy", "surname": "Bobson", "Level": "Basic"},
    {"Username": "Terry54", "Password": "OLETell54@", "name": "Terry", "surname": "Johnson", "Level": "Admin"}
]

with open("my_database.csv", "w", newline="") as file:
    fieldnames = list(test_data[0].keys())
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(test_data)





In [ ]:
db = Database.connect("my_database.csv")
data_test = {"Username": "timmy54", "Password": "Timmy54@", "name": "timmy", "surname": "tenders", "Level": "Admin"}

db.write_to_database(data_test)

Database my_database.csv ---> connected...


In [ ]:
db = Database.connect("my_database.csv")
user = User.create_admin(db)

Database my_database.csv ---> connected...
Create Username: greg54
Valid Username
Create Password: Greg54@
Valid Password
Input Your Name: greg
Input Your Surname: bob


**To use in a program by creating a main event loop**

In [ ]:
def main():

    db = Database.connect("my_database.csv")

    print("To create a new account type: 1 | To log in type: 0")
    login_choice = input("Choice: ")

    if login_choice == "1":
        # Create a new account
        create_new = User.create_admin(db)
        if create_new:
            print("Account created successfully!")
            name = create_new.user_data.get("name")
            print(f"Welcome, {name}!")

    elif login_choice == "0":
        # Log in
        logged_in_user = Authentication.login(db)
        if logged_in_user:
            # After successful login, continue with program logic
            name = logged_in_user.user_data.get("name")
            print(f"Welcome, {name}!")

            # Perform other tasks or operations using the logged_in_user instance

            # Continue with the rest of the program
    else:
        print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()

Database my_database.csv ---> connected...
To create a new account type: 1 | To log in type: 0
Choice: 0
Enter Username: gre54
Enter Password: Greg54@
You're In
Welcome, greg!


In [ ]:
import pandas as pd

df = pd.read_csv("my_database.csv")

df

,Username,Password,name,surname,Level
0,billy54,billyB54@,Billy,Bobson,Basic
1,Terry54,OLETell54@,Terry,Johnson,Admin
2,timmy54,Timmy54@,timmy,tenders,Admin
3,gre54,Greg54@,greg,bob,Admin
